In [13]:
import pandas as pd
import re
from scipy.stats.stats import pearsonr

In [14]:
df = pd.read_csv('charts.csv')[['date','artist']]
df['date'] = df['date'].apply(lambda x:pd.to_datetime(x).year)
df = df[df['date']>=2010]
df['artist'] = df['artist'].apply(lambda x:[ele.strip() for ele in re.split(',|featuring|&| x | \/ | with ',x.lower())])
tempdf = []
for ele in df.to_dict('records'):
    for artist in ele['artist']:
        tempdf.append({'date':ele['date'],'artist':artist})
df = pd.DataFrame(tempdf)
del tempdf
df['date'] = pd.Categorical(df['date'], categories=df['date'].unique())
df = df.groupby(['date','artist'],as_index=False).size()
df = df.iloc[::-1].reset_index(drop=True)
df["cumfreq"]=df.groupby(['artist'])['size'].cumsum(axis=0)
df['date'] = df['date'].apply(lambda x:int(x))

In [15]:
melon_df = pd.read_csv('melon_reviews.csv',lineterminator='\n')
del melon_df['Unnamed: 0']
melon_df['date'] = pd.to_datetime(melon_df['date']).apply(lambda x:x.year)
melon_df = melon_df[melon_df['date']<2022]
melon_df['artist'] = melon_df['artist'].apply(lambda x:x.lower())
melon_df

,id,artist,album,date,duration,description,thumbnail,views,score,genre,shirt,shirt_rgb
11,WxdzDxf0I0k,fire,Toolz,2021,PT8M35S,listen: <url> eternal home is seriously impre...,https://i.ytimg.com/vi/WxdzDxf0I0k/hqdefault.jpg,45812.0,7.0,"['blackened vapor', 'etc.']",other,"[(49, 32, 39), (202, 131, 109), (136, 125, 120)]"
12,dtNbOpGOjmw,juice wrld,Fighting Demons,2021,PT7M18S,listen: <url> fighting demons sadly isn't as ...,https://i.ytimg.com/vi/dtNbOpGOjmw/hqdefault.jpg,314677.0,4.0,['emo trap'],white_check,"[(181, 176, 171), (58, 50, 50), (100, 83, 83)]"
13,5l7sWQ9Z-9A,death's dynamic shroud.wmv,Faith in Persona,2021,PT7M54S,listen: <url> link for mixtape club members: <...,https://i.ytimg.com/vi/5l7sWQ9Z-9A/hqdefault.jpg,132345.0,8.0,"['vaporwave', 'dance-pop', 'glitch']",yellow_check,"[(163, 149, 120), (47, 33, 36), (79, 65, 65)]"
14,FVlDV15m854,l'rain,Fatigue,2021,PT5M55S,"listen: <url> late to the party, but yeah, th...",https://i.ytimg.com/vi/FVlDV15m854/hqdefault.jpg,55462.0,8.0,"['neo-psychedelic soul', 'sound collage']",other,"[(63, 50, 71), (183, 51, 63), (127, 120, 128)]"
15,yJywipy8DXw,catbite,Nice One,2021,PT4M47S,listen: <url> nice one! more rock reviews: <...,https://i.ytimg.com/vi/yJywipy8DXw/hqdefault.jpg,47284.0,7.0,['ska'],white_check,"[(190, 184, 181), (42, 28, 33), (85, 71, 72)]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2345,DuMUDldrG3g,the national,High Violet,2010,PT5M55S,review: <url> / <url>\r \r the national's late...,https://i.ytimg.com/vi/DuMUDldrG3g/hqdefault.jpg,60141.0,6.0,"[""http://bit.ly/c9kigf\r\n\r\nthe national's l...",other,"[(127, 56, 71), (145, 139, 133), (80, 34, 49)]"
2346,ySXryTlo9Ac,holy fuck,Latin,2010,PT5M40S,listen: <url> / <url> / <url>\r \r though it's...,https://i.ytimg.com/vi/ySXryTlo9Ac/hqdefault.jpg,11559.0,7.0,"[""http://bit.ly/bzrsr0\r\n\r\nthough it's not ...",other,"[(213, 206, 203), (74, 66, 66), (110, 98, 98)]"
2347,cndwH6byJnk,little women,Throat,2010,PT7M47S,listen: <url> let me start by saying this alb...,https://i.ytimg.com/vi/cndwH6byJnk/hqdefault.jpg,29826.0,9.0,NaN,NaN,NaN
2348,KCuamde9Atc,flying lotus,Cosmogramma,2010,PT7M52S,listen: <url> / <url> flying lotus' third alb...,https://i.ytimg.com/vi/KCuamde9Atc/hqdefault.jpg,168306.0,8.0,['http://bit.ly/apds3t'],other,"[(117, 69, 64), (152, 109, 100), (169, 116, 111)]"


In [16]:
df2 = pd.merge(melon_df, df[['date','artist','cumfreq']], on=['date', 'artist'])
df2 = df2.dropna()

In [22]:
import plotly.express as px
fig = px.scatter(x=df2['cumfreq'], y=df2['score'], range_x=(0,1000), title='"The Needle Drop" - Review Score vs Artist Popularity',trendline="ols",labels=dict(x="Weeks on Billboard Charts", y="score"))
fig.show()

In [18]:
print(pearsonr(df2['cumfreq'],df2['score']))

(-0.1274485575833631, 0.009261461774345761)
